In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [2]:
cataloguePath = "Catalogue.txt"
purchasesPath = "Purchases.txt"
catalogueRDD = sc.textFile(cataloguePath)
purchasesRDD = sc.textFile(purchasesPath)

output1 = "OutPart1"
output2 = "OutPart2"

In [6]:
# Part1 
# 2019/02/02-09:15:01,User20,ID1,50.99

def mapPurch2223(l):
    userId = l.split(",")[1]
    year = l.split(",")[0]
    if(year.startswith("2022/")):
        return (userId, (1, 0))
    else:
        return (userId, (0, 1))

userPurch2223 = purchasesRDD\
            .filter(lambda l: str(l).startswith("2022/") or str(l).startswith("2023/"))\
            .map(mapPurch2223)\
            .reduceByKey(lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1]))

In [8]:
maxs = userPurch2223\
    .values()\
    .reduce(lambda t1, t2: (max(t1[0], t2[0]), max(t1[1], t2[1])))

In [10]:
res1 = userPurch2223.filter(lambda i: i[1][0] == maxs[0] or i[1][1] == maxs[1]).keys()

In [11]:
res1.collect()

['User1', 'User3']

In [53]:
# Part2
# ID1,t-shirt-winter,Clothing,True

userPurchId2223 = purchasesRDD\
            .filter(lambda l: str(l).startswith("2022/") or str(l).startswith("2023/"))\
            .map(lambda l: (l.split(",")[2], 1))\
            .reduceByKey(lambda v1, v2: v1 + v2)\

# Pid, TotalAmount

PIDCategoryTot = userPurchId2223.join(catalogueRDD.map(lambda l: (l.split(",")[0],l.split(",")[2])))

In [54]:
maxCategory = PIDCategoryTot.map(lambda i: (i[1][1], i[1][0])).reduceByKey(lambda v1, v2: max(v1, v2))

In [55]:
maxCategory.collect()

[('Clothing', 5), ('Kitchen', 1)]

In [56]:
categoryNoItems = catalogueRDD.map(lambda l: (l.split(",")[2], "NoPurchases")).distinct().subtractByKey(maxCategory)

In [57]:
categoryNoItems.collect()

[('Home', 'NoPurchases')]

In [59]:
print("ciao")

ciao


In [58]:
# res3 = PIDCategoryTot\
    # .map(lambda i: (i[1][1], (i[0], i[1][0])))\
    # .join(maxCategory)\
    # .filter(lambda i: i[1][0][1] == i[1][1])\
    # .map(lambda i: (i[0], i[1][0][1]))